In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.component import Component
from pof.failure_mode import FailureMode
import pof.demo as demo
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.data.asset_data import SimpleFleet

In [2]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

paths = Paths()

# Population Data
file_path = paths.input_path + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR);

In [3]:
aml = AssetModelLoader(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp_data = aml.load(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp = Component.from_dict(comp_data["pole"])
comp.fleet_data = sfd

In [4]:
t_end = 1000
n_iterations = 10

In [5]:
# Turn failure modes on & off

failure_modes_on = ["termites"]
tasks_off = []

for fm in comp.fm.values():
    if fm.name not in failure_modes_on:
        fm.active = False
    else:
        fm.active = True

    for task in fm.tasks.values():
        if task.name in tasks_off:
            task.active = False
        else:
            task.active = True

In [6]:
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)

100%|██████████| 10/10 [00:00<00:00, 60.79it/s]


In [7]:
comp.expected_risk_cost_df();

In [8]:
fms_to_check = ["termites"]
tasks_to_check = ["conditional_failure", "functional_failure"]

In [9]:
dict_index = dict()

for fm_name, fm in comp.fm.items():
    if fm_name in fms_to_check:
        for index, timeline in fm._timelines.items():
            for task_name in tasks_to_check:
               if 0 in timeline[task_name]: 
                   dict_index[index] = {"fm":fm_name, "task":task_name}

dict_index;

In [10]:
comp.fm["termites"]._timelines[9];

In [11]:
comp.fm["termites"].indicators["safety_factor"]._timelines = None;

comp.fm["termites"].indicators["safety_factor"].__dict__;
